In [25]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## Helper function to extract the name of the author into a sepreate colum

In [43]:
def authors2author(name):
    tree = ET.parse(f'./data-cleaned/{name}.xml')    
    root = tree.getroot()
    for child in root:
        c = ET.Element("author")
        Names=child.findall('authors')[0].findall("author")[0].findall("name")
        text=""
        if len(Names)==1:
            text=Names[0].text
        else:
            for Name in Names:
                text+=Name.text+"/"
        c.text = text
        child.insert(2, c)
        child.remove(child.findall('authors')[0])
    tree=ET.ElementTree(root)
    with open(f'{name}M.xml', 'wb') as f:
        tree.write(f)

In [3]:
authors2author("kindle")
authors2author('recommendation')

# First recommendation and kindle data sets

In [3]:
df_kindle = pd.read_xml("kindleM.xml")

In [4]:
df_kindle

,id,title,author,rating,pages,price,language,publisher
0,kindle_1,How to be a Bartender: Get a Bartending Job Wi...,Dave Allred,4.0,171.0,9.0,English,None
1,kindle_2,Raise the Bar: An Action-Based Method for Maxi...,Jon Taffer,4.0,258.0,2.0,English,Amazon Publishing
2,kindle_3,The Professional Bartender's Handbook: A Recip...,Valerie Mellema,4.0,288.0,9.0,English,Atlantic Publishing Group Inc.; illustrated ed...
3,kindle_4,How to Double Your Tips,Jerry Minchey,4.0,36.0,2.0,English,Stony River Media
4,kindle_5,The Bartender's Bible: 1001 Mixed Drinks,Gary Regan,3.0,400.0,5.0,English,HarperCollins e-books
...,...,...,...,...,...,...,...,...
49179,kindle_49180,"Famous Gamblers, Poker History, and Texas Stories",Johnny Hughes,4.0,256.0,6.0,English,iUniverse
49180,kindle_49181,The Madness of March: Bonding and Betting with...,Alan Jay Zaremba,3.0,243.0,9.0,English,University of Nebraska Press
49181,kindle_49182,The Football Betting Bible,Stephen Ward,4.0,70.0,6.0,English,None
49182,kindle_49183,Winning on Betfair For Dummies,Alex Gowar,NaN,210.0,10.0,English,For Dummies; 2nd Revised edition edition


In [5]:
df_recommendation = pd.read_xml("recommendationM.xml")
df_recommendation

,id,title,author,year,publisher
0,recommendation_1,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,recommendation_2,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,recommendation_3,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,recommendation_4,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,recommendation_5,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271313,recommendation_271314,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271314,recommendation_271315,From One to One Hundred,Teri Sloat,1991,Dutton Books
271315,recommendation_271316,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271316,recommendation_271317,Republic (World's Classics),Plato,1996,Oxford University Press


In [202]:
Gold_df=pd.merge(df_recommendation, df_kindle,  left_on=['title','author'], right_on = ['title','author']) 
Gold_df=Gold_df.drop(['title','author','year','rating','pages','price','language'],1)
Gold_df["gold"] =  np.where(Gold_df['publisher_x'] == Gold_df['publisher_y'],True,False)
Gold_df

,id_x,publisher_x,id_y,publisher_y,gold
0,recommendation_19,Dell,kindle_97,Dell,True
1,recommendation_950,Doubleday Books,kindle_97,Dell,False
2,recommendation_4316,Trafalgar Square,kindle_97,Dell,False
3,recommendation_130480,Bantam Books,kindle_97,Dell,False
4,recommendation_178745,Random House Audio,kindle_97,Dell,False
...,...,...,...,...,...
4216,recommendation_270245,Benbella Books,kindle_47716,BenBella Books,False
4217,recommendation_270289,Perseus Publishing,kindle_21344,Basic Books; Second Edition edition,False
4218,recommendation_270699,Pomegranate,kindle_21910,None,False
4219,recommendation_270720,Berrett-Koehler Publishers,kindle_575,Berrett-Koehler Publishers; Second Edition edi...,False


In [203]:
Gold_T=Gold_df.drop(Gold_df[Gold_df['gold'] == False].index)
Gold_T

,id_x,publisher_x,id_y,publisher_y,gold
0,recommendation_19,Dell,kindle_97,Dell,True
51,recommendation_65,Ballantine Books,kindle_13763,Ballantine Books,True
53,recommendation_74343,Ballantine Books,kindle_13763,Ballantine Books,True
154,recommendation_514,Atlantic Monthly Press,kindle_28945,Atlantic Monthly Press,True
162,recommendation_648,Scribner,kindle_18205,Scribner,True
...,...,...,...,...,...
4143,recommendation_241369,See Sharp Press,kindle_22023,See Sharp Press,True
4147,recommendation_241896,Hyperion,kindle_8426,Hyperion,True
4152,recommendation_243493,HarperBusiness,kindle_7708,HarperBusiness,True
4161,recommendation_249702,Penguin Books,kindle_34571,Penguin Books,True


### This will be part of the True part of the GS (20%)

### Now, Handling the corner cases by hand

In [204]:
Gold_H=Gold_df.drop(Gold_df[Gold_df['gold'] == True].index)
Gold_H=Gold_H.sort_values(by=['publisher_y'], ascending=False)
Gold_H.reset_index( inplace=True)
Gold_H.drop('index',1,inplace=True)
Gold_H

,id_x,publisher_x,id_y,publisher_y,gold
0,recommendation_37262,Push,kindle_26687,namelos,False
1,recommendation_116505,Critics Choice Paperbacks,kindle_33088,iUniverse,False
2,recommendation_197924,Kensington Pub Corp (Mm),kindle_23553,ePublishing Works!,False
3,recommendation_235742,Pinnacle Books,kindle_32947,ePublishing Works!,False
4,recommendation_65224,HarperTorch,kindle_4346,ePublishing Works!,False
...,...,...,...,...,...
3922,recommendation_251496,Bantam Books,kindle_29019,None,False
3923,recommendation_258325,Dover Publications,kindle_10136,None,False
3924,recommendation_258326,Dover Publications,kindle_25924,None,False
3925,recommendation_260517,Fawcett Books,kindle_34980,None,False


In [73]:
Gold_H.loc[1:380].to_csv('recommendation_kindle_H.csv')

# After that some of the data were evaluated by hand and then read again

In [205]:
Gold_H_D = pd.read_csv("recommendation_kindle_H_done.csv")
Gold_H_D.drop('Unnamed: 0',1,inplace=True)
Gold_H_D

,id_x,publisher_x,id_y,publisher_y,gold
0,recommendation_116505,Critics Choice Paperbacks,kindle_33088,iUniverse,False
1,recommendation_197924,Kensington Pub Corp (Mm),kindle_23553,ePublishing Works!,False
2,recommendation_235742,Pinnacle Books,kindle_32947,ePublishing Works!,False
3,recommendation_65224,HarperTorch,kindle_4346,ePublishing Works!,False
4,recommendation_9970,Ivy Books,kindle_3150,eKensington,False
...,...,...,...,...,...
279,recommendation_35794,Tor Books,kindle_24643,Tor Books; Reprint edition,True
280,recommendation_128972,Putnam Pub Group,kindle_35184,Tor Books; Reprint edition,False
281,recommendation_6059,Berkley Publishing Group,kindle_35184,Tor Books; Reprint edition,False
282,recommendation_164863,Tor Books,kindle_39013,Tor Books; First Edition edition,True


## This will be represent the corner cases (30%)

## Now creating the non-mathces 

In [206]:
Gold_false_df=pd.merge(df_recommendation, df_kindle,  left_on=['title'], right_on = ['title']) 
Gold_false_df=Gold_false_df.drop(['title','year','rating','pages','price','language'],1)
Gold_false_df["gold"] =  np.where(((Gold_false_df['author_x'] == Gold_false_df['author_y'])|(Gold_false_df['publisher_x'] == Gold_false_df['publisher_y'])),True,False)
Gold_false_df=Gold_false_df.drop(Gold_false_df[Gold_false_df['gold'] == True].index)
Gold_false_df=Gold_false_df.sort_values(by=['publisher_y',"author_y"], ascending=False)
Gold_false_df.reset_index( inplace=True)
Gold_false_df.drop('index',1,inplace=True)
Gold_false_df

,id_x,author_x,publisher_x,id_y,author_y,publisher_y,gold
0,recommendation_162435,Charles Ryan,Harper Mass Market Paperbacks (Mm),kindle_38744,Philip S. Donlay,ipicturebooks; First edition,False
1,recommendation_99396,Georg W. Hegel,Dover Publications,kindle_5337,Georg Hegel,iap,False
2,recommendation_158618,Betty Neels,Harlequin,kindle_4066,Patricia Kiyono,esKape Press,False
3,recommendation_24991,Sara Paretsky,Dell Publishing Company,kindle_43791,Melanie Phillips,emBooks; 1 edition,False
4,recommendation_29823,Sara Paretsky,Bantam Dell Pub Group,kindle_43791,Melanie Phillips,emBooks; 1 edition,False
...,...,...,...,...,...,...,...
4064,recommendation_120390,Jodi Picoult,Berkley Publishing Group,kindle_16096,Alessandra Thomas,None,False
4065,recommendation_239841,Marie Bourdon,Kensington Publishing Corporation,kindle_16096,Alessandra Thomas,None,False
4066,recommendation_267250,Jodi Picoult,Putnam Pub Group,kindle_16096,Alessandra Thomas,None,False
4067,recommendation_52461,Bertrand Russell,Oxford University Press,kindle_7486,Albert Einstein,None,False


In [14]:
Gold_false_df.loc[1:500].to_csv('recommendation_kindle_F_H.csv')

# After that some of the data were evaluated by hand and then read again

In [207]:
Gold_F = pd.read_csv("recommendation_kindle_F_H_done.csv")
Gold_F.drop('Unnamed: 0',1,inplace=True)
Gold_F

,id_x,author_x,publisher_x,id_y,author_y,publisher_y,gold
0,recommendation_99396,Georg W. Hegel,Dover Publications,kindle_5337,Georg Hegel,iap,False
1,recommendation_158618,Betty Neels,Harlequin,kindle_4066,Patricia Kiyono,esKape Press,False
2,recommendation_24991,Sara Paretsky,Dell Publishing Company,kindle_43791,Melanie Phillips,emBooks; 1 edition,False
3,recommendation_29823,Sara Paretsky,Bantam Dell Pub Group,kindle_43791,Melanie Phillips,emBooks; 1 edition,False
4,recommendation_56116,Julie Garwood,Pocket,kindle_43791,Melanie Phillips,emBooks; 1 edition,False
...,...,...,...,...,...,...,...
495,recommendation_146353,Stella Cameron,Harper Mass Market Paperbacks (Mm),kindle_1635,J.P. Smith,Thomas & Mercer,False
496,recommendation_1074,LISA GARDNER,Bantam,kindle_1378,J. Carson Black,Thomas & Mercer,False
497,recommendation_49448,LISA GARDNER,Bantam,kindle_1378,J. Carson Black,Thomas & Mercer,False
498,recommendation_71598,LISA GARDNER,Random House Audio,kindle_1378,J. Carson Black,Thomas & Mercer,False


In [208]:
Gold_T.drop(['publisher_x','publisher_y'],1,inplace=True)
Gold_H_D.drop(['publisher_x','publisher_y'],1,inplace=True)
Gold_F.drop(['publisher_x','publisher_y','author_x','author_y'],1,inplace=True)

In [211]:
gs_recommendation_kindle=Gold_T.append(Gold_H_D).append(Gold_F)
gs_recommendation_kindle.reset_index( inplace=True)
gs_recommendation_kindle.drop('index',1,inplace=True)
gs_recommendation_kindle


,id_x,id_y,gold
0,recommendation_19,kindle_97,True
1,recommendation_65,kindle_13763,True
2,recommendation_74343,kindle_13763,True
3,recommendation_514,kindle_28945,True
4,recommendation_648,kindle_18205,True
...,...,...,...
1073,recommendation_146353,kindle_1635,False
1074,recommendation_1074,kindle_1378,False
1075,recommendation_49448,kindle_1378,False
1076,recommendation_71598,kindle_1378,False


# Then we have our gold standard with 60% non-matches and 40% matches

In [41]:
gs_recommendation_kindle.to_csv('./gs/gs_recommendation_kindle.csv', header=False, index=False)

# 440 of 1078 of the records are matched (40%) and 638 from 1078 are non-matched (60%)

# Second kindle and goodreads data sets

In [44]:
authors2author("goodreads")

In [45]:
df_goodreads = pd.read_xml("goodreadsM.xml")
df_goodreads

,id,title,author,rating,pages,genres
0,goodreads_1,The Hunger Games,Suzanne Collins,4,374.0,NaN
1,goodreads_2,Harry Potter and the Order of the Phoenix,J.K. Rowling/Mary GrandPré/,4,870.0,NaN
2,goodreads_3,To Kill a Mockingbird,Harper Lee,4,324.0,NaN
3,goodreads_4,Pride and Prejudice,Jane Austen/Anna Quindlen/Mrs. Oliphant/George...,4,279.0,NaN
4,goodreads_5,Twilight,Stephenie Meyer,3,498.0,NaN
...,...,...,...,...,...,...
54295,goodreads_54296,Taking the Field: A Fan's Quest to Run the Tea...,Howard Megdal,3,256.0,NaN
54296,goodreads_54297,"The Baseball Talmud: Koufax, Greenberg, and th...",Howard Megdal,3,256.0,NaN
54297,goodreads_54298,"Wilpon's Folly - The Story of a Man, His Fortu...",Howard Megdal,3,NaN,NaN
54298,goodreads_54299,He Wanted the Moon: The Madness and Medical Ge...,Mimi Baird/Eve Claxton/,3,272.0,NaN


In [46]:
df_goodreads_2=df_goodreads.drop(['genres','rating'],1)
df_kindle_2=df_kindle.drop(['price','language','rating','publisher'],1)

In [47]:
Gold_df=pd.merge(df_goodreads_2, df_kindle_2,  left_on=['title','pages'], right_on = ['title','pages']) 
Gold_df["gold"] =  np.where(Gold_df['author_x'] == Gold_df['author_y'],True,False)
Gold_df.drop(["title","pages"],1,inplace=True)
Gold_df

,id_x,author_x,id_y,author_y,gold
0,goodreads_34,John Steinbeck,kindle_3245,John Steinbeck,True
1,goodreads_70,Sylvia Plath/Reinhard Kaiser/,kindle_3612,Sylvia Plath,False
2,goodreads_31103,Sylvia Plath,kindle_3612,Sylvia Plath,True
3,goodreads_126,Umberto Eco/William Weaver/Seán Barrett/,kindle_9696,Umberto Eco,False
4,goodreads_335,Ann Patchett,kindle_1083,Ann Patchett,True
...,...,...,...,...,...
492,goodreads_53653,Jodi Lynn Anderson,kindle_45411,Jodi Lynn Anderson,True
493,goodreads_53671,Derek Sanderson/Kevin Shea/,kindle_22268,Derek Sanderson,False
494,goodreads_53733,Neil Young,kindle_757,Neil Young,True
495,goodreads_53739,Kenneth Grahame,kindle_25913,Kenneth Grahame,True


In [48]:
Gold_T=Gold_df.drop(Gold_df[Gold_df['gold'] == False].index)
Gold_T

,id_x,author_x,id_y,author_y,gold
0,goodreads_34,John Steinbeck,kindle_3245,John Steinbeck,True
2,goodreads_31103,Sylvia Plath,kindle_3612,Sylvia Plath,True
4,goodreads_335,Ann Patchett,kindle_1083,Ann Patchett,True
5,goodreads_344,Marie Lu,kindle_2493,Marie Lu,True
6,goodreads_12528,Marie Lu,kindle_2493,Marie Lu,True
...,...,...,...,...,...
490,goodreads_53398,Mark Z. Danielewski,kindle_41250,Mark Z. Danielewski,True
492,goodreads_53653,Jodi Lynn Anderson,kindle_45411,Jodi Lynn Anderson,True
494,goodreads_53733,Neil Young,kindle_757,Neil Young,True
495,goodreads_53739,Kenneth Grahame,kindle_25913,Kenneth Grahame,True


### This will be part of the True part of the GS (20%)

## This will be represent the corner cases (30%)

In [49]:
Gold_H=Gold_df.drop(Gold_df[Gold_df['gold'] == True].index)
Gold_H=Gold_H.sort_values(by=['author_y'], ascending=False)
Gold_H.reset_index( inplace=True)
Gold_H.drop('index',1,inplace=True)
Gold_H

,id_x,author_x,id_y,author_y,gold
0,goodreads_33803,Zilpha Keatley Snyder/Alton Raible/,kindle_27124,Zilpha Keatley Snyder,False
1,goodreads_37474,Yotam Ottolenghi/Sami Tamimi/Richard Learoyd/,kindle_4164,Yotam Ottolenghi,False
2,goodreads_34027,Yōko Ogawa/Stephen Snyder/,kindle_46514,Yoko Ogawa,False
3,goodreads_26011,Nicola Haken,kindle_22733,William Cope Moyers,False
4,goodreads_35785,Whittaker Chambers/William F. Buckley Jr./Robe...,kindle_11818,Whittaker Chambers,False
...,...,...,...,...,...
150,goodreads_16463,Angelo Dirks/Leland Dirks/,kindle_3538,Angelo Dirks,False
151,goodreads_50176,Angela Ackerman/Becca Puglisi/,kindle_4931,Angela Ackerman,False
152,goodreads_2317,Amie Kaufman/Meagan Spooner/,kindle_21065,Amie Kaufman,False
153,goodreads_34625,Alison G. Bailey,kindle_27442,Alison Bailey,False


In [148]:
Gold_H.to_csv('goodreads_kindle_H.csv')

# After that some of the data were evaluated by hand and then read again

In [53]:
Gold_H_D = pd.read_csv('goodreads_kindle_H_done.csv')
Gold_H_D.drop('Unnamed: 0',1,inplace=True)
Gold_H_D

,id_x,author_x,id_y,author_y,gold
0,goodreads_33803,Zilpha Keatley Snyder/Alton Raible/,kindle_27124,Zilpha Keatley Snyder,True
1,goodreads_37474,Yotam Ottolenghi/Sami Tamimi/Richard Learoyd/,kindle_4164,Yotam Ottolenghi,True
2,goodreads_34027,Yōko Ogawa/Stephen Snyder/,kindle_46514,Yoko Ogawa,True
3,goodreads_26011,Nicola Haken,kindle_22733,William Cope Moyers,False
4,goodreads_35785,Whittaker Chambers/William F. Buckley Jr./Robe...,kindle_11818,Whittaker Chambers,True
...,...,...,...,...,...
150,goodreads_16463,Angelo Dirks/Leland Dirks/,kindle_3538,Angelo Dirks,True
151,goodreads_50176,Angela Ackerman/Becca Puglisi/,kindle_4931,Angela Ackerman,True
152,goodreads_2317,Amie Kaufman/Meagan Spooner/,kindle_21065,Amie Kaufman,True
153,goodreads_34625,Alison G. Bailey,kindle_27442,Alison Bailey,True


In [54]:

gs_goodreads_kindle=Gold_T.drop(['author_x','author_y'],1).append(Gold_H_D.drop(Gold_H_D[Gold_H_D['gold'] == False].index).drop(['author_x','author_y'],1)).append(Gold_H_D.drop(Gold_H_D[Gold_H_D['gold'] == True].index).drop(['author_x','author_y'],1))
gs_goodreads_kindle.reset_index( inplace=True)
gs_goodreads_kindle.drop('index',1,inplace=True)
gs_goodreads_kindle

,id_x,id_y,gold
0,goodreads_34,kindle_3245,True
1,goodreads_31103,kindle_3612,True
2,goodreads_335,kindle_1083,True
3,goodreads_344,kindle_2493,True
4,goodreads_12528,kindle_2493,True
...,...,...,...
492,goodreads_19138,kindle_19046,False
493,goodreads_22985,kindle_7462,False
494,goodreads_33014,kindle_1067,False
495,goodreads_15558,kindle_26077,False


### We noticed most of the data are matches, so we will repeate the process with some tweeks

### This function to create additional columns to help out filtering the data

In [58]:
def authors2authorTry(name):
    tree = ET.parse(f'./data-cleaned/{name}.xml')    
    root = tree.getroot()
    for child in root:
        Names=child.findall('authors')[0].findall("author")[0].findall("name")
        y = ET.Element("author")
        y.text=Names[0].text
        child.insert(1, y)
        if len(Names)>1:
            k = ET.Element("Mul")
            k.text='TRUE'
            child.insert(3, k)
        text=""
        if len(Names)==1:
            text=Names[0].text
        else:
            for Name in Names:
                text+=Name.text+"/"
        c = ET.Element("Allauthors")
        c.text = text
        child.insert(1, c)
        child.remove(child.findall('authors')[0])
    tree=ET.ElementTree(root)
    with open(f'{name}MT.xml', 'wb') as f:
        tree.write(f)


In [73]:
authors2authorTry("goodreads")
df_g = pd.read_xml("goodreadsMT.xml")
df_g

,id,Allauthors,author,title,rating,pages,genres,Mul
0,goodreads_1,Suzanne Collins,Suzanne Collins,The Hunger Games,4,374.0,NaN,None
1,goodreads_2,J.K. Rowling/Mary GrandPré/,J.K. Rowling,Harry Potter and the Order of the Phoenix,4,870.0,NaN,TRUE
2,goodreads_3,Harper Lee,Harper Lee,To Kill a Mockingbird,4,324.0,NaN,None
3,goodreads_4,Jane Austen/Anna Quindlen/Mrs. Oliphant/George...,Jane Austen,Pride and Prejudice,4,279.0,NaN,TRUE
4,goodreads_5,Stephenie Meyer,Stephenie Meyer,Twilight,3,498.0,NaN,None
...,...,...,...,...,...,...,...,...
54295,goodreads_54296,Howard Megdal,Howard Megdal,Taking the Field: A Fan's Quest to Run the Tea...,3,256.0,NaN,None
54296,goodreads_54297,Howard Megdal,Howard Megdal,"The Baseball Talmud: Koufax, Greenberg, and th...",3,256.0,NaN,None
54297,goodreads_54298,Howard Megdal,Howard Megdal,"Wilpon's Folly - The Story of a Man, His Fortu...",3,NaN,NaN,None
54298,goodreads_54299,Mimi Baird/Eve Claxton/,Mimi Baird,He Wanted the Moon: The Madness and Medical Ge...,3,272.0,NaN,TRUE


In [74]:
df_g=df_g.drop(['genres','rating'],1)
df_k=df_kindle.drop(['price','language','rating','publisher'],1)

In [75]:
G_df=pd.merge(df_g, df_k,  left_on=['title'], right_on = ['title']) 
G_df["gold"] =  np.where(((G_df['pages_x'] == G_df['pages_y'])|(G_df['author_x'] == G_df['author_y'])),True,False)
G_df

,id_x,Allauthors,author_x,title,pages_x,Mul,id_y,author_y,pages_y,gold
0,goodreads_1,Suzanne Collins,Suzanne Collins,The Hunger Games,374.0,None,kindle_5559,Suzanne Collins,387.0,True
1,goodreads_14937,Suzanne Collins,Suzanne Collins,The Hunger Games,454.0,None,kindle_5559,Suzanne Collins,387.0,True
2,goodreads_21678,Suzanne Collins,Suzanne Collins,The Hunger Games,374.0,None,kindle_5559,Suzanne Collins,387.0,True
3,goodreads_22255,Suzanne Collins,Suzanne Collins,The Hunger Games,436.0,None,kindle_5559,Suzanne Collins,387.0,True
4,goodreads_4,Jane Austen/Anna Quindlen/Mrs. Oliphant/George...,Jane Austen,Pride and Prejudice,279.0,TRUE,kindle_9845,Jane Austen,258.0,True
...,...,...,...,...,...,...,...,...,...,...
5459,goodreads_54040,"James Wesley, Rawles","James Wesley, Rawles",How to Survive the End of the World as We Know...,336.0,None,kindle_19680,JamesWesley Rawles,340.0,False
5460,goodreads_54043,Richard Price,Richard Price,The Wanderers,NaN,None,kindle_21113,Richard Price,260.0,True
5461,goodreads_54051,Kazuo Ishiguro,Kazuo Ishiguro,When We Were Orphans,336.0,None,kindle_9715,Kazuo Ishiguro,352.0,True
5462,goodreads_54134,Nadine Gordimer,Nadine Gordimer,The Conservationist,267.0,None,kindle_47622,Nadine Gordimer,271.0,True


In [77]:
G_df=G_df.drop(G_df[G_df['gold'] == True].index)
G_df=G_df.drop(G_df[G_df['Mul'] != "TRUE"].index)
G_df=G_df.drop(['author_x','title','pages_x','Mul','pages_y'],1)
G_df.reset_index( inplace=True)
G_df.drop('index',1,inplace=True)
G_df

,id_x,Allauthors,id_y,author_y,gold
0,goodreads_34803,Elie Wiesel/Marion Wiesel/,kindle_37949,Anya Bast,False
1,goodreads_14,Emily Brontë/Richard J. Dunn/David Timson/Char...,kindle_25002,Emily Bronte,False
2,goodreads_14,Emily Brontë/Richard J. Dunn/David Timson/Char...,kindle_25220,Emily Bronte,False
3,goodreads_15034,Emily Brontë/Pauline Nestor/Lucasta Miller/,kindle_25002,Emily Bronte,False
4,goodreads_15034,Emily Brontë/Pauline Nestor/Lucasta Miller/,kindle_25220,Emily Bronte,False
...,...,...,...,...,...
411,goodreads_51673,Marcus Annaeus Lucanus/Matthew Fox/Ethan Adams/,kindle_25990,Mark Millar,False
412,goodreads_52118,J.D. Robb/Nora Roberts/,kindle_13514,J. D. Robb,False
413,goodreads_52570,J.D. Robb/Nora Roberts/,kindle_4586,J. D. Robb,False
414,goodreads_53094,Antonio J. Méndez/Matt Baglio/Dylan Baker/,kindle_22008,Antonio Mendez,False


### We merged based on title and choose the books with different pages and the first author is different, Now we need to make sure by hand that these entites are really different

In [236]:
G_df.loc[1:250].to_csv('goodreads_kindle_H2.csv')

In [135]:
after_R = pd.read_csv("goodreads_kindle_H2_done.csv")
after_R.drop('Unnamed: 0',1,inplace=True)
after_R.drop(['Allauthors','author_y'],1,inplace=True)
after_R

,id_x,id_y,gold
0,goodreads_18600,kindle_25018,False
1,goodreads_27951,kindle_25018,False
2,goodreads_37011,kindle_25913,False
3,goodreads_296,kindle_15580,False
4,goodreads_24147,kindle_15580,False
...,...,...,...
169,goodreads_26643,kindle_47337,False
170,goodreads_44254,kindle_47337,False
171,goodreads_10119,kindle_47322,False
172,goodreads_7972,kindle_5327,False


### Now we have our corner cases

## Now creating the non-mathces (50%)

In [167]:
Gold_F=pd.merge(df_g, df_k,  left_on=['title'], right_on = ['title'])
Gold_F['author_y'] = Gold_F['author_y'].str.lower()
Gold_F['author_x'] = Gold_F['author_x'].str.lower()
Gold_F["gold"] =  np.where(((Gold_F['pages_x'] == Gold_F['pages_y'])|(Gold_F['author_x'].str[0] == Gold_F['author_y'].str[0])),True,False)
Gold_F=Gold_F.drop(Gold_F[Gold_F['gold'] == True].index)
Gold_F=Gold_F.drop(Gold_F[Gold_F['Mul'] == "TRUE"].index)
Gold_F=Gold_F.drop(['Allauthors','title','pages_x','Mul','pages_y'],1)
Gold_F=Gold_F.sort_values(by=['author_y'], ascending=True)
Gold_F.reset_index( inplace=True)
Gold_F.drop('index',1,inplace=True)
Gold_F

,id_x,author_x,id_y,author_y,gold
0,goodreads_52733,christopher pike,kindle_41132,a. lee martinez,False
1,goodreads_12972,francette phal,kindle_41132,a. lee martinez,False
2,goodreads_9548,walter dean myers,kindle_41132,a. lee martinez,False
3,goodreads_4896,frank e. peretti,kindle_41132,a. lee martinez,False
4,goodreads_19452,lylah james,kindle_28536,a.k. alexander,False
...,...,...,...,...,...
1484,goodreads_34874,barbara haworth-attard,kindle_10192,willow cross,False
1485,goodreads_14408,morgan rice,kindle_47931,wodke hawkinson,False
1486,goodreads_15667,catherine coulter,kindle_11047,yvie towers,False
1487,goodreads_4275,christopher shields,kindle_10958,zilpha keatley snyder,False


In [136]:
Gold_F_Final=Gold_F.loc[0:670].drop(['author_x','author_y'],1)
Gold_F_Final

,id_x,id_y,gold
0,goodreads_52733,kindle_41132,False
1,goodreads_12972,kindle_41132,False
2,goodreads_9548,kindle_41132,False
3,goodreads_4896,kindle_41132,False
4,goodreads_19452,kindle_28536,False
...,...,...,...
666,goodreads_15640,kindle_32791,False
667,goodreads_13350,kindle_44032,False
668,goodreads_7490,kindle_9506,False
669,goodreads_2928,kindle_9506,False


In [142]:
gs_goodreads_kindle_final=gs_goodreads_kindle.append(after_R).append(Gold_F_Final)
gs_goodreads_kindle_final.reset_index(inplace=True)
gs_goodreads_kindle_final.drop('index',1,inplace=True)
gs_goodreads_kindle_final

,id_x,id_y,gold
0,goodreads_34,kindle_3245,True
1,goodreads_31103,kindle_3612,True
2,goodreads_335,kindle_1083,True
3,goodreads_344,kindle_2493,True
4,goodreads_12528,kindle_2493,True
...,...,...,...
1337,goodreads_15640,kindle_32791,False
1338,goodreads_13350,kindle_44032,False
1339,goodreads_7490,kindle_9506,False
1340,goodreads_2928,kindle_9506,False


### Finally We have our gold standard

In [144]:
gs_goodreads_kindle_final.to_csv('./gs/gs_goodreads_kindle.csv', header=False, index=False)

# 496 of 1342 are Matches (36%) and 846 of 1342 are non-matches(63%)

# Third: goodreads and recommendation datasets

In [145]:
df_recommendation = pd.read_xml("recommendationM.xml")
df_recommendation

,id,title,author,year,publisher
0,recommendation_1,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,recommendation_2,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,recommendation_3,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,recommendation_4,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,recommendation_5,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271313,recommendation_271314,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271314,recommendation_271315,From One to One Hundred,Teri Sloat,1991,Dutton Books
271315,recommendation_271316,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271316,recommendation_271317,Republic (World's Classics),Plato,1996,Oxford University Press


In [146]:
df_goodreadsMT = pd.read_xml("goodreadsMT.xml")
df_goodreadsMT

,id,Allauthors,author,title,rating,pages,genres,Mul
0,goodreads_1,Suzanne Collins,Suzanne Collins,The Hunger Games,4,374.0,NaN,None
1,goodreads_2,J.K. Rowling/Mary GrandPré/,J.K. Rowling,Harry Potter and the Order of the Phoenix,4,870.0,NaN,TRUE
2,goodreads_3,Harper Lee,Harper Lee,To Kill a Mockingbird,4,324.0,NaN,None
3,goodreads_4,Jane Austen/Anna Quindlen/Mrs. Oliphant/George...,Jane Austen,Pride and Prejudice,4,279.0,NaN,TRUE
4,goodreads_5,Stephenie Meyer,Stephenie Meyer,Twilight,3,498.0,NaN,None
...,...,...,...,...,...,...,...,...
54295,goodreads_54296,Howard Megdal,Howard Megdal,Taking the Field: A Fan's Quest to Run the Tea...,3,256.0,NaN,None
54296,goodreads_54297,Howard Megdal,Howard Megdal,"The Baseball Talmud: Koufax, Greenberg, and th...",3,256.0,NaN,None
54297,goodreads_54298,Howard Megdal,Howard Megdal,"Wilpon's Folly - The Story of a Man, His Fortu...",3,NaN,NaN,None
54298,goodreads_54299,Mimi Baird/Eve Claxton/,Mimi Baird,He Wanted the Moon: The Madness and Medical Ge...,3,272.0,NaN,TRUE


In [171]:
df_T=pd.merge(df_recommendation, df_goodreadsMT,  left_on=['title'], right_on = ['title'])
df_T.drop(['title','pages','year','publisher','Allauthors','rating','genres'],1,inplace=True)
df_T["gold"] =  np.where(((df_T['author_x'] == df_T['author_y'])),True,False)
df_T.drop_duplicates(subset=['id_x'],inplace=True,keep='last')
df_T.drop_duplicates(subset=['id_y'],inplace=True,keep='last')
df_T.drop(df_T[df_T['gold'] == False].index,inplace=True)
df_T.drop(df_T[df_T['Mul'] == "TRUE"].index,inplace=True)
df_T.reset_index(inplace=True)
df_T.drop('index',1,inplace=True)
df_T

,id_x,author_x,id_y,author_y,Mul,gold
0,recommendation_265819,Amy Tan,goodreads_39164,Amy Tan,None,True
1,recommendation_140757,Michael Crichton,goodreads_6475,Michael Crichton,None,True
2,recommendation_101234,Mark Salzman,goodreads_42505,Mark Salzman,None,True
3,recommendation_197237,Harper Lee,goodreads_34674,Harper Lee,None,True
4,recommendation_18497,Barbara Kingsolver,goodreads_7157,Barbara Kingsolver,None,True
...,...,...,...,...,...,...
4322,recommendation_269715,Anita Diamant,goodreads_24641,Anita Diamant,None,True
4323,recommendation_269756,William Blake,goodreads_53454,William Blake,None,True
4324,recommendation_270161,Parke Godwin,goodreads_26854,Parke Godwin,None,True
4325,recommendation_270339,Isaac Bashevis Singer,goodreads_52753,Isaac Bashevis Singer,None,True


In [172]:
Gold_T=df_T.loc[0:332].drop(['author_x','author_y','Mul'],1)
Gold_T

,id_x,id_y,gold
0,recommendation_265819,goodreads_39164,True
1,recommendation_140757,goodreads_6475,True
2,recommendation_101234,goodreads_42505,True
3,recommendation_197237,goodreads_34674,True
4,recommendation_18497,goodreads_7157,True
...,...,...,...
328,recommendation_171646,goodreads_49758,True
329,recommendation_29141,goodreads_20025,True
330,recommendation_194273,goodreads_8638,True
331,recommendation_81330,goodreads_3050,True


### This will be part of the True part of the GS (20%)

## Now creating the corner cases (30%) 

In [177]:
df=pd.merge(df_recommendation, df_goodreadsMT,  left_on=['title'], right_on = ['title'])
df

,id_x,title,author_x,year,publisher,id_y,Allauthors,author_y,rating,pages,genres,Mul
0,recommendation_6,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,goodreads_927,Amy Tan,Amy Tan,3,416.0,NaN,None
1,recommendation_6,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,goodreads_39164,Amy Tan,Amy Tan,3,415.0,NaN,None
2,recommendation_1430,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,goodreads_927,Amy Tan,Amy Tan,3,416.0,NaN,None
3,recommendation_1430,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,goodreads_39164,Amy Tan,Amy Tan,3,415.0,NaN,None
4,recommendation_27681,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,goodreads_927,Amy Tan,Amy Tan,3,416.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...
32820,recommendation_270885,Radiance,Anne Maybury,1980,Ace Books,goodreads_21680,Grace Draven,Grace Draven,4,297.0,NaN,None
32821,recommendation_270885,Radiance,Anne Maybury,1980,Ace Books,goodreads_46350,Catherynne M. Valente,Catherynne M. Valente,3,429.0,NaN,None
32822,recommendation_270993,The Broken Sword,Northam Anderson,1981,Del Rey Books,goodreads_49628,Poul Anderson,Poul Anderson,3,274.0,NaN,None
32823,recommendation_271207,Pathfinder,James Fenimore Cooper,1982,Signet Book,goodreads_24738,Orson Scott Card,Orson Scott Card,3,657.0,NaN,None


In [178]:
df.drop_duplicates(subset=['id_x'],inplace=True,keep='first')
df.drop_duplicates(subset=['id_y'],inplace=True,keep='first')
df["gold"] =  np.where(df['author_x'] == df['author_y'],True,False)
df.drop(df[df['gold'] == True].index,inplace=True)
df=df.drop(df[df['Mul'] != "TRUE"].index)
df.drop(['title','pages','year','publisher','author_y','rating','genres','Mul'],1,inplace=True)
df=df.sort_values(by=['author_x'], ascending=False)
df.reset_index( inplace=True)
df.drop('index',1,inplace=True)
df

,id_x,author_x,id_y,Allauthors,gold
0,recommendation_55650,ZILPHA KEATLEY SNYDER,goodreads_44716,Zilpha Keatley Snyder/Alton Raible/,False
1,recommendation_18817,Yukio Pseud Mishima,goodreads_11051,Yukio Mishima/Meredith Weatherby/,False
2,recommendation_43755,Yoko Kawashawa Watkins,goodreads_26788,Yoko Kawashima Watkins/Jean Fritz/,False
3,recommendation_57663,Yashar Kemal,goodreads_29282,Yaşar Kemal/Edouard Roditi/,False
4,recommendation_41284,Yan Mo,goodreads_39681,Mo Yan/Howard Goldblatt/,False
...,...,...,...,...,...
789,recommendation_17421,A. N. Roquelaure,goodreads_15926,A.N. Roquelaure/Anne Rice/,False
790,recommendation_124861,A. N. Roquelaure,goodreads_7788,A.N. Roquelaure/Anne Rice/,False
791,recommendation_77830,A. N. Roquelaure,goodreads_6265,A.N. Roquelaure/Anne Rice/,False
792,recommendation_209016,A. Jarry,goodreads_45370,Alfred Jarry/Barbara Wright/Franciszka Themerson/,False


In [315]:
df.loc[:500].to_csv('goodreads_recommendation_H.csv')

# After that some of the data were evaluated by hand and then read again

In [190]:
Gold_C = pd.read_csv('goodreads_recommendation_H_done.csv')
Gold_C.drop('Unnamed: 0',1,inplace=True)
Gold_C=Gold_C.drop(['author_x','Allauthors'],1)
Gold_C

,id_x,id_y,gold
0,recommendation_55650,goodreads_44716,True
1,recommendation_18817,goodreads_11051,True
2,recommendation_43755,goodreads_26788,True
3,recommendation_57663,goodreads_29282,True
4,recommendation_41284,goodreads_39681,True
...,...,...,...
496,recommendation_125920,goodreads_52118,True
497,recommendation_35603,goodreads_52070,True
498,recommendation_78424,goodreads_2030,False
499,recommendation_57059,goodreads_1342,True


## Now creating non-matches (50%)

In [187]:
df_F=pd.merge(df_recommendation, df_goodreadsMT,  left_on=['title'], right_on = ['title'])
df_F.drop(['title','pages','year','publisher','Allauthors','rating','genres'],1,inplace=True)
df_F['author_y'] = df_F['author_y'].str.lower()
df_F['author_x'] = df_F['author_x'].str.lower()
df_F["gold"] =  np.where(((df_F['author_x'].str[0] == df_F['author_y'].str[0])),True,False)
df_F.drop_duplicates(subset=['id_x'],inplace=True,keep='last')
df_F.drop_duplicates(subset=['id_y'],inplace=True,keep='last')
df_F.drop(df_F[df_F['gold'] == True].index,inplace=True)
df_F.drop(df_F[df_F['Mul'] == "TRUE"].index,inplace=True)
df_F.reset_index( inplace=True)
df_F.drop('index',1,inplace=True)
df_F

,id_x,author_x,id_y,author_y,Mul,gold
0,recommendation_186677,elie wiesel,goodreads_1814,john grisham,None,False
1,recommendation_201348,warren ellis,goodreads_29759,robert a. heinlein,None,False
2,recommendation_131125,william johnstone,goodreads_28766,christopher farnsworth,None,False
3,recommendation_195707,james patterson,goodreads_45646,sally john,None,False
4,recommendation_219116,cynthia strickland,goodreads_2862,elle kennedy,None,False
...,...,...,...,...,...,...
2340,recommendation_269529,elizabeth elliott,goodreads_41619,morgan rice,None,False
2341,recommendation_270398,brenda k. jernigan,goodreads_14719,johanna lindsey,None,False
2342,recommendation_270808,fumiko enchi,goodreads_14264,evangeline anderson,None,False
2343,recommendation_270885,anne maybury,goodreads_46350,catherynne m. valente,None,False


In [188]:
Gold_F=df_F.loc[1:830].drop(['author_x','author_y','Mul'],1)
Gold_F

,id_x,id_y,gold
1,recommendation_201348,goodreads_29759,False
2,recommendation_131125,goodreads_28766,False
3,recommendation_195707,goodreads_45646,False
4,recommendation_219116,goodreads_2862,False
5,recommendation_178709,goodreads_43801,False
...,...,...,...
826,recommendation_198913,goodreads_6287,False
827,recommendation_229859,goodreads_12768,False
828,recommendation_150192,goodreads_30658,False
829,recommendation_33819,goodreads_17246,False


In [194]:
gs_goodreads_recommendation_final=Gold_F.append(Gold_T).append(Gold_C)
gs_goodreads_recommendation_final.reset_index( inplace=True)
gs_goodreads_recommendation_final.drop(['index'],1,inplace=True)
gs_goodreads_recommendation_final

,id_x,id_y,gold
0,recommendation_201348,goodreads_29759,False
1,recommendation_131125,goodreads_28766,False
2,recommendation_195707,goodreads_45646,False
3,recommendation_219116,goodreads_2862,False
4,recommendation_178709,goodreads_43801,False
...,...,...,...
1659,recommendation_125920,goodreads_52118,True
1660,recommendation_35603,goodreads_52070,True
1661,recommendation_78424,goodreads_2030,False
1662,recommendation_57059,goodreads_1342,True


### Finally We have our gold standard

In [212]:
gs_goodreads_recommendation_final.to_csv('./gs/gs_recommendation_goodreads.csv', header=False, index=False)

# 607 of 1664 are Matches (36%) and 1057 are non-matches(63%)